# Phishing Website Detection 

Step -1 : Data preprocessing 

This dataset contains few website links (Some of them are legitimate websites and a few are fake websites)

Pre-Processing the data before building a model and also Extracting the features from the data based on certain conditions

In [64]:
#importing numpy and pandas which are required for data pre-processing
import numpy as np
import pandas as pd

In [65]:
#Loading the data
test_data = pd.read_csv("dataset.csv") 
#loading only 100 samples (art websites data)

In [66]:
test_data.head()

,URL,Target
0,https://locking-app-adverds.000webhostapp.com/...,yes
1,https://mxtoolbox.com/public/upgradev2.aspx?ps...,no
2,http://parolishop.com.br/questionreviews.htm,yes
3,http://innovate.ieee.org/innovate/35675?lt=xpl...,no
4,http://ietbhaddal.edu.in/bb.mobile/mobile/inde...,yes


We need to split the data according to parts of the URL

A typical URL could have the form http://www.example.com/index.html, which indicates a protocol (http), a hostname (www.example.com), and a file name (index.html).

Detailed explanation about URL:  1.https://doepud.co.uk/blog/anatomy-of-a-url                                                    
                                 2.https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL                  
                                 3.https://techwelkin.com/understanding-the-components-and-structure-of-a-url
                                        
                                            

In [67]:
test_data['URL'].str.split("://").head() #Here we divided the protocol from the entire URL. but need it to be divided it 
                                                 #seperate column

0    [https, locking-app-adverds.000webhostapp.com/...
1    [https, mxtoolbox.com/public/upgradev2.aspx?ps...
2        [http, parolishop.com.br/questionreviews.htm]
3    [http, innovate.ieee.org/innovate/35675?lt=xpl...
4    [http, ietbhaddal.edu.in/bb.mobile/mobile/inde...
Name: URL, dtype: object








refer to this link for splitting of data --> https://apassionatechie.wordpress.com/2018/02/24/how-do-i-split-a-string-into-several-columns-in-a-dataframe-with-pandas-python/

In [68]:
seperation_of_protocol = test_data['URL'].str.split("://",expand = True) #expand argument in the split method will give you a new column

In [69]:
seperation_of_protocol.head()

,0,1,2
0,https,locking-app-adverds.000webhostapp.com/payment-...,None
1,https,mxtoolbox.com/public/upgradev2.aspx?ps=03eaebd...,None
2,http,parolishop.com.br/questionreviews.htm,None
3,http,innovate.ieee.org/innovate/35675?lt=xpllg_xpl_...,None
4,http,ietbhaddal.edu.in/bb.mobile/mobile/index.php,None


In [70]:
type(seperation_of_protocol)

pandas.core.frame.DataFrame

In [71]:
seperation_domain_name = seperation_of_protocol[1].str.split("/",1,expand = True) #split(seperator,no of splits according to seperator(delimiter),expand)

In [72]:
type(seperation_domain_name)

pandas.core.frame.DataFrame

In [73]:
seperation_domain_name.columns=["domain_name","address"] #renaming columns of data frame

In [74]:
seperation_domain_name.head()

,domain_name,address
0,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...
1,mxtoolbox.com,public/upgradev2.aspx?ps=03eaebd0-170a-4142-a7...
2,parolishop.com.br,questionreviews.htm
3,innovate.ieee.org,innovate/35675?lt=xpllg_xpl_bold728x90_q3.16_f...
4,ietbhaddal.edu.in,bb.mobile/mobile/index.php


In [75]:
#Concatenation of data frames
splitted_data = pd.concat([seperation_of_protocol[0],seperation_domain_name],axis=1)


In [76]:
splitted_data.columns = ['protocol','domain_name','address']

In [78]:
splitted_data['is_phished'] = pd.Series(test_data['Target'], index=splitted_data.index)

In [79]:
splitted_data.head()

,protocol,domain_name,address,is_phished
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes
1,https,mxtoolbox.com,public/upgradev2.aspx?ps=03eaebd0-170a-4142-a7...,no
2,http,parolishop.com.br,questionreviews.htm,yes
3,http,innovate.ieee.org,innovate/35675?lt=xpllg_xpl_bold728x90_q3.16_f...,no
4,http,ietbhaddal.edu.in,bb.mobile/mobile/index.php,yes


Domain name column can be further sub divided into domain_names as well as sub_domain_names 

Similarly, address column can also be further sub divided into path,query_string,file..................

In [40]:
type(splitted_data)

pandas.core.frame.DataFrame

### Features Extraction


Feature-1

1.Long URL to Hide the Suspicious Part

If the length of the URL is greater than or equal 54 characters then the URL classified as phishing


0 --- indicates legitimate

1 --- indicates Phishing

2 --- indicates Suspicious

In [80]:
def long_url(l):
    l= str(l)
    """This function is defined in order to differntiate website based on the length of the URL"""
    if len(l) < 54:
        return 0
    elif len(l) >= 54 and len(l) <= 75:
        return 2
    return 1

In [81]:
#Applying the above defined function in order to divide the websites into 3 categories
splitted_data['long_url'] = test_data['URL'].apply(long_url) 


In [82]:
#Will show the results only the websites which are legitimate according to above condition as 0 is legitimate website
splitted_data[splitted_data.long_url == 0] 

,protocol,domain_name,address,is_phished,long_url
2,http,parolishop.com.br,questionreviews.htm,yes,0
4,http,ietbhaddal.edu.in,bb.mobile/mobile/index.php,yes,0
5,http,www.vivianmaier.com,film-finding-vivian-maier/,no,0
7,http,effbot.org,zone/python-list.htm,no,0
8,http,apply.ycombinator.com,,no,0
9,http,www.hbo.com,movies/furious-7,no,0
10,https,customer.xfinity.com,#/billing,no,0
16,http,fearlesssecurity.com,,no,0
25,http,code.google.com,p/pylevenshtein/,no,0
26,http,linkedin.com,,no,0


Feature-2

2.URL’s having “@” Symbol

Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

IF {Url Having @ Symbol→ Phishing
    Otherwise→ Legitimate }


0 --- indicates legitimate

1 --- indicates Phishing


In [83]:
def have_at_symbol(l):
    """This function is used to check whether the URL contains @ symbol or not"""
    if "@" in str(l):
        return 1
    return 0
    

In [84]:
splitted_data['having_@_symbol'] = test_data['URL'].apply(have_at_symbol)

In [85]:
splitted_data

,protocol,domain_name,address,is_phished,long_url,having_@_symbol
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0
1,https,mxtoolbox.com,public/upgradev2.aspx?ps=03eaebd0-170a-4142-a7...,no,1,0
2,http,parolishop.com.br,questionreviews.htm,yes,0,0
3,http,innovate.ieee.org,innovate/35675?lt=xpllg_xpl_bold728x90_q3.16_f...,no,1,0
4,http,ietbhaddal.edu.in,bb.mobile/mobile/index.php,yes,0,0
5,http,www.vivianmaier.com,film-finding-vivian-maier/,no,0,0
6,http,smokesonstate.com,gssss/gssss/27560bea886c96f3aac10ec0ff159568,yes,2,0
7,http,effbot.org,zone/python-list.htm,no,0,0
8,http,apply.ycombinator.com,,no,0,0
9,http,www.hbo.com,movies/furious-7,no,0,0


Feature-3

3.Redirecting using “//”

The existence of “//” within the URL path means that the user will be redirected to another website.
An example of such URL’s is: “http://www.legitimate.com//http://www.phishing.com”. 
We examine the location where the “//” appears. 
We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. 
However, if the URL employs “HTTPS” then the “//” should appear in seventh position.

IF {ThePosition of the Last Occurrence of "//" in the URL > 7→ Phishing
    
    Otherwise→ Legitimate

0 --- indicates legitimate

1 --- indicates Phishing


In [86]:
def redirection(l):
    """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
    if "//" in str(l):
        return 1
    return 0

In [87]:
splitted_data['redirection_//_symbol'] = seperation_of_protocol[1].apply(redirection)

In [88]:
splitted_data.head()

,protocol,domain_name,address,is_phished,long_url,having_@_symbol,redirection_//_symbol
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0,0
1,https,mxtoolbox.com,public/upgradev2.aspx?ps=03eaebd0-170a-4142-a7...,no,1,0,0
2,http,parolishop.com.br,questionreviews.htm,yes,0,0,0
3,http,innovate.ieee.org,innovate/35675?lt=xpllg_xpl_bold728x90_q3.16_f...,no,1,0,0
4,http,ietbhaddal.edu.in,bb.mobile/mobile/index.php,yes,0,0,0


Feature-4

4.Adding Prefix or Suffix Separated by (-) to the Domain

The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name
so that users feel that they are dealing with a legitimate webpage. 

For example http://www.Confirme-paypal.com/.
    
IF {Domain Name Part Includes (−) Symbol → Phishing
    
    Otherwise → Legitimate
    
1 --> indicates phishing

0 --> indicates legitimate
    

In [89]:
def prefix_suffix_seperation(l):
    if '-' in str(l):
        return 1
    return 0

In [90]:
splitted_data['prefix_suffix_seperation'] = seperation_domain_name['domain_name'].apply(prefix_suffix_seperation)

In [91]:
splitted_data.head()

,protocol,domain_name,address,is_phished,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0,0,1
1,https,mxtoolbox.com,public/upgradev2.aspx?ps=03eaebd0-170a-4142-a7...,no,1,0,0,0
2,http,parolishop.com.br,questionreviews.htm,yes,0,0,0,0
3,http,innovate.ieee.org,innovate/35675?lt=xpllg_xpl_bold728x90_q3.16_f...,no,1,0,0,0
4,http,ietbhaddal.edu.in,bb.mobile/mobile/index.php,yes,0,0,0,0


Feature - 5

5. Sub-Domain and Multi Sub-Domains

The legitimate URL link has two dots in the URL since we can ignore typing “www.”. 
If the number of dots is equal to three then the URL is classified as “Suspicious” since it has one sub-domain.
However, if the dots are greater than three it is classified as “Phishy” since it will have multiple sub-domains

0 --- indicates legitimate

1 --- indicates Phishing

2 --- indicates Suspicious


In [92]:
def sub_domains(l):
    l= str(l)
    if l.count('.') < 3:
        return 0
    elif l.count('.') == 3:
        return 2
    return 1

In [93]:
splitted_data['sub_domains'] = splitted_data['domain_name'].apply(sub_domains)

In [94]:
splitted_data

,protocol,domain_name,address,is_phished,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0,0,1,0
1,https,mxtoolbox.com,public/upgradev2.aspx?ps=03eaebd0-170a-4142-a7...,no,1,0,0,0,0
2,http,parolishop.com.br,questionreviews.htm,yes,0,0,0,0,0
3,http,innovate.ieee.org,innovate/35675?lt=xpllg_xpl_bold728x90_q3.16_f...,no,1,0,0,0,0
4,http,ietbhaddal.edu.in,bb.mobile/mobile/index.php,yes,0,0,0,0,0
5,http,www.vivianmaier.com,film-finding-vivian-maier/,no,0,0,0,0,0
6,http,smokesonstate.com,gssss/gssss/27560bea886c96f3aac10ec0ff159568,yes,2,0,0,0,0
7,http,effbot.org,zone/python-list.htm,no,0,0,0,0,0
8,http,apply.ycombinator.com,,no,0,0,0,0,0
9,http,www.hbo.com,movies/furious-7,no,0,0,0,0,0


In [96]:
splitted_data.to_csv(r'C:\Users\komal\Documents\PERSONAL STUFF\WORK\Phishing-Website-Detection\dataset3.csv',header= True)